In [6]:
from bs4 import BeautifulSoup
import requests
import html5lib
import re
import pandas as pd
import numpy as np

In [7]:
cookies = {
    'ccsid': '668-8707266-8901325',
    '__qca': 'P0-1002348396-1661184067604',
    'p': 'ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd',
    'likely_has_account': 'true',
    'srb_8': '0_wl',
    'locale': 'en',
    'u': 'oTJu21ixofG3HvrbsDKN1hg8e8FfKW_FUT1R8ZnFKULdyBce',
    'csm-sid': '696-1026276-2871101',
    'logged_out_browsing_page_count': '2',
    '_session_id2': '56f88bb472b69394ced13b9599f84511',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9,id;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'ccsid=668-8707266-8901325; __qca=P0-1002348396-1661184067604; p=ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd; likely_has_account=true; srb_8=0_wl; locale=en; u=oTJu21ixofG3HvrbsDKN1hg8e8FfKW_FUT1R8ZnFKULdyBce; csm-sid=696-1026276-2871101; logged_out_browsing_page_count=2; _session_id2=56f88bb472b69394ced13b9599f84511',
    'If-None-Match': 'W/"7a8fffe16d5f0300c16298a353c8fe08"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not?A_Brand";v="8", "Chromium";v="108", "Google Chrome";v="108"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

# params = {
#     'page': '1',
# }

# response = requests.get('https://www.goodreads.com/shelf/show/horror', cookies=cookies, headers=headers)


In [8]:
def clean_publish_year(raw_year):
    if type(raw_year) is None:
        return "Missing"
    result = re.search(r"(\b\d+)$", raw_year)
    if result == None:
        return 0
    else:
        return result.group(1)

def clean_rating(raw_rating):
    if type(raw_rating) is None:
        return 0
    result = re.search(r"(\b\d+.\b\d+)", raw_rating)
    if result == None:
        return 0
    else:
        return result.group(1)

def clean_raters(raw_raters):
    if type(raw_raters) is None:
        return 0
    result = re.search(r"((?:\d+,?\d+)*) ratings", raw_raters)
    if result == None:
        return 0
    else:
        return result.group(1)

# def clean_readers(raw_readers):
#     result = re.search(r"(\b\d+)", raw_readers)
#     if result == None:
#         return "None"
#     else:
#         return result.group(1)

def solve_nonetype_error(value):
    if value is not None:
        return value.text.strip()
    else:
        return "Missing"



In [9]:
books = []
for page in range(1, 26):
    response = requests.get("https://www.goodreads.com/shelf/show/to-read?page=" + str(page), cookies = cookies, headers=headers)
    html_soup = BeautifulSoup(response.text, 'html5lib')
    book_container = html_soup.find_all('div', class_ = "elementList")
    for book in book_container:
        bookTitle = solve_nonetype_error(book.find('a', class_ = "bookTitle"))
       
        bookAuthor =solve_nonetype_error(book.find('a', class_ = "authorName"))

        # After(use comments)
        year = clean_publish_year(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))
        rating = clean_rating(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))
        raters = str(clean_raters(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))).replace(",", "")
        int(raters)
        # # readerNum = clean_readers(book.find('span', class_ = "greyText statistic").text.strip())
        
        # Before (use comments)
        # year = solve_nonetype_error(book.find('span', class_ = "greyText smallText"))
        # rating = solve_nonetype_error(book.find('span', class_ = "greyText smallText"))
        # raters = solve_nonetype_error(book.find('span', class_ = "greyText smallText"))
        # readerNum = book.find('span', class_ = "greyText statistic").text.strip()


        book_data = [bookTitle, bookAuthor, year, rating, raters]
        books.append(book_data)
books = pd.DataFrame(books, columns=["Title", "Author", "Year", "Rating", "Raters"])
books.head()

,Title,Author,Year,Rating,Raters
0,To Kill a Mockingbird (Paperback),Harper Lee,1960,4.27,5489146
1,1984 (Paperback),George Orwell,1949,4.19,4014982
2,Pride and Prejudice (Paperback),Jane Austen,1813,4.28,3784309
3,The Book Thief (Hardcover),Markus Zusak,2006,4.39,2273552
4,The Fault in Our Stars (Hardcover),John Green,2012,4.16,4562043


In [10]:
books.to_csv("TopBestCookies.csv", index = False)
